In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math

from tensorflow.python.framework import ops
from sklearn.model_selection import train_test_split

train_path = './data/train.csv'
test_path = './data/test.csv'

In [2]:
class get_data:
    def __init__(self, path, is_train=True):
        data = pd.read_csv(path)
        if is_train:
            ts, vs = train_test_split(data, test_size=0.1, random_state=2)
            self.ts_f, self.ts_l = self.split_f_l(ts)
            self.vs_f, self.vs_l = self.split_f_l(vs)
            self.dum_train_val()
        else:
            self.feature = data.values
#         if is_train:
#             self.feature = data.drop('label', axis=1)
#             self.label = data['label']
#         else:
#             self.feature = data
#         del data
        
#         self.feature = self.feature.values
#         if is_train:
#             self.label = pd.get_dummies(self.label).values
#             self.ts_f, self.ts_l, self.vs_f, self_vs_l = self.split_t_v()
            
    def split_f_l(self, data):
        f = data.drop('label', axis=1)
        l = data['label']
        return f, l
    
    def dum_train_val(self):
        self.ts_f = self.ts_f.values
        self.ts_l = pd.get_dummies(self.ts_l).values
        self.vs_f = self.vs_f.values
        self.vs_l = pd.get_dummies(self.vs_l).values

In [9]:
class create_model:
    def __init__(
        self,
        ts_f,
        ts_l,
        vs_f,
        vs_l,
        learning_rate=1e-3,
        epochs=30,
        batch_size=64
    ):
        self.ts_f = ts_f
        self.ts_l = ts_l
        self.vs_f = vs_f
        self.vs_l = vs_l
        
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        
        # reset computation graph
        ops.reset_default_graph()
        self.x, self.y, self.prediction_layer = self.buildCNN()
        # loss, optim
        self.loss, self.optim = self.optimization()
        # acc
        self.correct_prediction = tf.equal(
            tf.argmax(self.prediction_layer, 1),
            tf.argmax(self.y, 1)
        )
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, 'float'))
        
        # initialize
        self.init = tf.global_variables_initializer()
        
    def buildCNN(self):
        x = tf.placeholder(tf.float32, shape=(None, self.ts_f.shape[1]), name='x')
        y = tf.placeholder(tf.float32, shape=(None, self.ts_l.shape[1]), name='y')
        
        # flatten the input
        # 1st layer
        m = tf.layers.dense(
            x,
            units=256,
            activation='relu',
            kernel_initializer=tf.contrib.layers.xavier_initializer(),
            bias_initializer=tf.zeros_initializer()
        )
        m = tf.layers.dropout(inputs=m, rate=0.4)
        
        # 2nd layer
        m = tf.layers.dense(
            m,
            units=128,
            activation='relu',
            kernel_initializer=tf.contrib.layers.xavier_initializer(),
            bias_initializer=tf.zeros_initializer()
        )
        # m = tf.layers.dropout(inputs=m, rate=0.4)
        
        # 3rd layer
        m = tf.layers.dense(
            m, units=64,
            activation='relu', 
            kernel_initializer=tf.contrib.layers.xavier_initializer(),
            bias_initializer=tf.zeros_initializer()
        )
        # m = tf.layers.dropout(inputs=m, rate=0.4)
        
        # 4th layer
        m = tf.layers.dense(
            m,
            units=32,
            activation='relu', 
            kernel_initializer=tf.contrib.layers.xavier_initializer(),
            bias_initializer=tf.zeros_initializer()
        )
        # m = tf.layers.dropout(inputs=m, rate=0.4)
        
        # 5th layer
        m = tf.layers.dense(
            m,
            units=16,
            activation='relu', 
            kernel_initializer=tf.contrib.layers.xavier_initializer(),
            bias_initializer=tf.zeros_initializer()
        )
        # m = tf.layers.dropout(inputs=m, rate=0.4)
        
        # output layer
        prediction = tf.layers.dense(
            m,
            units=self.ts_l.shape[1],
            name="p", 
            kernel_initializer=tf.contrib.layers.xavier_initializer(),
            bias_initializer=tf.zeros_initializer()
        )
        
        return x, y, prediction
    
    def optimization(self):
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=self.prediction_layer, labels=self.y))
        optim = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(loss)
        return loss, optim
    
    def train(self):
        with tf.Session() as sess:
            sess.run(self.init)  #initializes the variables created
            for epoch in range(self.epochs):
                epoch_cost = 0
                epoch_acc = 0
                num_minibatches = math.ceil(self.ts_f.shape[0] / self.batch_size)
                minibatches = self.random_mini_batches()
                for minibatch in minibatches:
                    (minibatch_x, minibatch_y) = minibatch
                    _, minibatch_cost, p, minibatch_acc  = sess.run([self.optim, self.loss, self.prediction_layer, self.accuracy], feed_dict = {self.x: minibatch_x, self.y: minibatch_y})
                    # print("pred shape: ", p)
                    epoch_cost += minibatch_cost / num_minibatches
                    epoch_acc += minibatch_acc / num_minibatches

                print("cost after epoch %i :  %.3f" % (epoch + 1, epoch_cost), end="")
                print("  train accuracy   :  %.3f" % epoch_acc)
                print("  cv accuracy   :  %.3f" % (self.accuracy.eval({self.x: self.vs_f, self.y: self.vs_l})))
            print("network trained")
            
#             predicts = tf.argmax(pred, 1).eval({x:x_test})
#             probs = tf.nn.softmax(pred, 1).eval({x: x_test})
#             print("test shape: ", x_test.shape)
#             print("predicts shape: ", predicts.shape)
#             print("predicts val: ", predicts)
#             return predicts, probs
    
    def random_mini_batches(self):
        m = self.ts_f.shape[0]
        mini_batches = []

        #shuffle x and y
        permutation = list(np.random.permutation(m))
        shuffled_x = self.ts_f[permutation]
        shuffled_y = self.ts_l[permutation]

        #partition
        num_complete_minibatches = math.ceil(m / self.batch_size)
        for k in range(0, num_complete_minibatches):
            mini_batch_x = shuffled_x[k*self.batch_size : k*self.batch_size + self.batch_size]
            mini_batch_y = shuffled_y[k*self.batch_size : k*self.batch_size + self.batch_size]
            mini_batch = (mini_batch_x, mini_batch_y)
            mini_batches.append(mini_batch)
        return mini_batches

    def predict(self, test_df):
        preds = tf.argmax(self.pred, 1).eval({x: test_df})
        probs = tf.nn.softmax(self.pred, 1).eval({x: test_df})
        
        preds = preds.reshape(-1,1)
        preds_df = pd.DataFrame(preds, columns=['Label'])
        preds_df['ImageID'] = preds_df.index + 1
        self.submission = preds_df[preds_df.columns[::-1]]
        
    def save_res(self, path):
        self.submission.to_csv(path, index=False, header=True)

In [10]:
train = get_data(train_path)
# train.ts_l.shape

model = create_model(
    train.ts_f,
    train.ts_l,
    train.vs_f,
    train.vs_l
)

model.train()

cost after epoch 1 :  0.423  train accuracy   :  0.313
  cv accuracy   :  0.487
cost after epoch 2 :  0.109  train accuracy   :  0.813
  cv accuracy   :  0.887
cost after epoch 3 :  0.055  train accuracy   :  0.936
  cv accuracy   :  0.935
cost after epoch 4 :  0.042  train accuracy   :  0.952
  cv accuracy   :  0.946
cost after epoch 5 :  0.034  train accuracy   :  0.958
  cv accuracy   :  0.950
cost after epoch 6 :  0.024  train accuracy   :  0.966
  cv accuracy   :  0.960
cost after epoch 7 :  0.018  train accuracy   :  0.973
  cv accuracy   :  0.962
cost after epoch 8 :  0.014  train accuracy   :  0.978
  cv accuracy   :  0.966
cost after epoch 9 :  0.012  train accuracy   :  0.982
  cv accuracy   :  0.960


KeyboardInterrupt: 